In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
#mouting google drive to the colab ws
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!unzip '/content/drive/MyDrive/Learning from Networks Project/archive.zip' #unzip the data folder

Archive:  /content/drive/MyDrive/Learning from Networks Project/archive.zip
   creating: archive/
  inflating: __MACOSX/._archive      
  inflating: archive/links_small.csv  
  inflating: __MACOSX/archive/._links_small.csv  
  inflating: archive/links.csv       
  inflating: __MACOSX/archive/._links.csv  
  inflating: archive/credits.csv     
  inflating: __MACOSX/archive/._credits.csv  
  inflating: archive/movies_metadata.csv  
  inflating: __MACOSX/archive/._movies_metadata.csv  
  inflating: archive/ratings.csv     
  inflating: __MACOSX/archive/._ratings.csv  
  inflating: archive/ratings_small.csv  
  inflating: __MACOSX/archive/._ratings_small.csv  
  inflating: archive/keywords.csv    
  inflating: __MACOSX/archive/._keywords.csv  


In [4]:
%cd /content/archive/

/content/archive


In [5]:
movies_metadata = pd.read_csv("movies_metadata.csv") #ids are str
movies_metadata = movies_metadata.drop(
    ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id', 'original_language', 'overview',
     'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'runtime',
     'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'], axis=1)
credits_metadata = pd.read_csv("credits.csv")
credits_metadata = credits_metadata.drop(['crew'], axis=1) #ids are int
credits_metadata.id=credits_metadata.id.astype(str) #equalizing id types for merging two table according to the ids
merged_df = pd.merge(movies_metadata, credits_metadata, on="id") #merge dataframes on ids
merged_df=merged_df.dropna().reset_index()  #deleting rows which contains empty cells, and then indexing lines again

#deleting lines which contains less than 3 actors in the movie because we are analysing top three actors for each movie or show
for i in range(len(merged_df)):
     if len(ast.literal_eval(merged_df["cast"][i]))<3:
         merged_df.drop(index=i, inplace=True)

merged_df=merged_df.reset_index()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
#adding new columns for 3 stars of each movie or show and adding to our dataframe 
star1L,star2L,star3L=[],[],[]
for i in range(len(merged_df)):
    cast = ast.literal_eval(merged_df["cast"][i])
    star1L.append(cast[0]["name"])
    star2L.append(cast[1]["name"])
    star3L.append(cast[2]["name"])

finalDf=merged_df.assign(firstStar=star1L,secondStar=star2L,thirdStar=star3L)

In [16]:
finalDf.drop(["level_0","index"], axis=1) #deleting index and the label because these columns are unnecessary

,id,original_title,revenue,cast,firstStar,secondStar,thirdStar
0,862,Toy Story,373554033.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",Tom Hanks,Tim Allen,Don Rickles
1,8844,Jumanji,262797249.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",Robin Williams,Jonathan Hyde,Kirsten Dunst
2,15602,Grumpier Old Men,0.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",Walter Matthau,Jack Lemmon,Ann-Margret
3,31357,Waiting to Exhale,81452156.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",Whitney Houston,Angela Bassett,Loretta Devine
4,11862,Father of the Bride Part II,76578911.0,"[{'cast_id': 1, 'character': 'George Banks', '...",Steve Martin,Diane Keaton,Martin Short
...,...,...,...,...,...,...,...
40866,30840,Robin Hood,0.0,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...",Patrick Bergin,Uma Thurman,David Morrissey
40867,439050,رگ خواب,0.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...",Leila Hatami,Kourosh Tahami,Elham Korda
40868,111109,Siglo ng Pagluluwal,0.0,"[{'cast_id': 1002, 'character': 'Sister Angela...",Angel Aquino,Perry Dizon,Hazel Orencio
40869,67758,Betrayal,0.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",Erika Eleniak,Adam Baldwin,Julie du Page


In [13]:
%cd /content

/content


In [18]:
#final data will be used for analysing if the starts acting in a movie effect to gross or not 
path = '/content/drive/MyDrive/Learning from Networks Project/data.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  finalDf.to_csv(f)